In [17]:
import os

In [18]:
%pwd

'c:\\Users\\abhis\\Downloads\\DS Interview Prep\\textsummarizer'

In [3]:
os.chdir("../")

In [19]:
### Entity (Return Type of a function)
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [20]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml,create_directories

In [21]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [22]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [41]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config =  config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )


        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
        return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


    def convert(self):
        dataset_samsun=load_from_disk("C:/Users/abhis/Downloads/DS Interview Prep/textsummarizer/artifacts/data_ingestion/samsum_dataset")
        dataset_samsun_pt=dataset_samsun.map(self.convert_examples_to_features,batched=True)
        dataset_samsun_pt.save_to_disk(r"C:/Users/abhis/Downloads/DS Interview Prep/textsummarizer/artifacts/data_transformation/samsum_dataset")

In [42]:
try:
    config=ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-06-18 11:00:35,118:INFO:common:yaml file: C:\Users\abhis\Downloads\DS Interview Prep\textsummarizer\config\config.yaml loaded successfully]
[2023-06-18 11:00:35,120:INFO:common:yaml file: params.yaml loaded successfully]
[2023-06-18 11:00:35,123:INFO:common:created directory at: artifacts]
[2023-06-18 11:00:35,125:INFO:common:created directory at: artifacts/data_transformation]


In [16]:
os.listdir("C:/Users/abhis/Downloads/DS Interview Prep/textsummarizer/artifacts/")

['data_ingestion', 'data_validation', 'transformation']

In [27]:
os.getcwd()

'c:\\Users\\abhis\\Downloads\\DS Interview Prep\\textsummarizer'